In [1]:
import pandas as pd
import numpy as np

# combined data - 2 tables

In [2]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [3]:
filename_leadercareerlink = "leadercareerlink.xlsx"

In [4]:
filename_leaderjoblink = "leaderjoblink.xlsx"

In [5]:
filename_joborglink = "joborglink.xlsx"

# combined data - 1 cleaning/cleaning step 2 - career/

In [6]:
path_cleaning = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/"

In [7]:
subpath_2_1 = "2.1 career_undivided_unparsed_uncoded/"
subpath_2_2 = "2.2 career_divided_unparsed_uncoded/"
subpath_2_3 = "2.3 joborglink/"

In [8]:
filein_careerlink = "career_link_undivided_unparsed_uncoded.xlsx"
careerlink = pd.read_excel(path_cleaning + subpath_2_1 + filein_careerlink)
careerlink.shape

(12985, 4)

In [9]:
filein_careerjobs = "career_jobs_unparsed_uncoded.xlsx"
careerjobs = pd.read_excel(path_cleaning + subpath_2_2 + filein_careerjobs)
careerjobs.shape

(7590, 3)

In [96]:
# hand coded MultipleSubstrings, NotJob
# update these variables in careers
# but [CareerString,CareerDateString_2022] are mismatched
# ignore CareerDateString_2022. only match on CareerString

filein_multiple = "joborglink_1_mergeold_mail.xlsx"
jol1 = pd.read_excel(path_cleaning + subpath_2_3 + filein_multiple)
jol1.shape

(7592, 14)

In [151]:
filein_singlesub = "joborglink_2_substr_single_1tocode_4Mail_Lee.xlsx"
singlesub = pd.read_excel(path_cleaning + subpath_2_3 + filein_singlesub)
singlesub.shape

(5897, 9)

In [123]:
# multisub should have 1695 unique CareerString. 1756 are the expanded CareerSubstrings
filein_multisub = "joborglink_2_substr_multi2_1tocode_Mail_Lee.xlsx"
multisub = pd.read_excel(path_cleaning + subpath_2_3 + filein_multisub)
multisub.shape

(1756, 11)

In [11]:
filein_jol_7 = "joborglink_7_2.xlsx"
jol7 = pd.read_excel(path_cleaning + subpath_2_3 + filein_jol_7)
jol7.shape

(1425, 4)

# 1. leadercareerlink 

In [12]:
# from career_link_undivided_unparsed_uncoded
careerlink.head(3)

,LeaderID,CareerStringID_2015,CareerString,CareerDateString_2022
0,리선권,NaN,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,NaN,故 김정일 국가장의위원회 위원,2011.12
2,리영수,NaN,근로단체부장 해임 * 후임: 리일환,2014


In [13]:
select_columns = ["LeaderID","CareerString","CareerDateString_2022"]
careerlink = careerlink[select_columns]

In [14]:
careerlink[careerlink.duplicated(keep=False)]

,LeaderID,CareerString,CareerDateString_2022
241,김일본,1950.12 민족보위성 부상 겸 문화부 부장,NaN
242,김일본,1950.12 민족보위성 부상 겸 문화부 부장,NaN
247,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
248,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
249,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
...,...,...,...
5468,현영철,"2012. 7 인민군 총참모장, 인민군 차수, 당 중앙군사위원회 부위원장",NaN
5488,리철만,2013. 4 내각 부총리 겸 농업상,NaN
5489,리철만,2013. 4 내각 부총리 겸 농업상,NaN
5490,리무영,2013. 4 내각 부총리 겸 화학공업상,NaN


In [15]:
leadercareerlink = careerlink.drop_duplicates(keep="first")
leadercareerlink.shape

(12617, 3)

In [16]:
leadercareerlink.head(5)

,LeaderID,CareerString,CareerDateString_2022
0,리선권,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,故 김정일 국가장의위원회 위원,2011.12
2,리영수,근로단체부장 해임 * 후임: 리일환,2014
3,리설주,금수산궁전 개관식 참석시 동행,2013.08
4,리원일,노동성 상(유임),1999.02


In [17]:
# export leadercareerlink to tables
# leadercareerlink.to_excel(path_tables+filename_leadercareerlink,index=False)

# 2. joborglink

### 2.1 - careers - isolate careers from leadercareerlink

In [18]:
select_columns = ["CareerString","CareerDateString_2022"]
careers = leadercareerlink[select_columns]
careers.shape

(12617, 2)

In [19]:
careers[careers.duplicated(keep=False)].shape

(5125, 2)

In [20]:
careers = careers.drop_duplicates(keep="first")
careers.shape

(8045, 2)

In [21]:
careers[careers.duplicated(["CareerString","CareerDateString_2022"],keep=False)]

,CareerString,CareerDateString_2022


### 2.2 - careers0 - add IsJob
- compare careers with career_jobs_unparsed_uncoded
- add IsJob=False for all rows
- recode IsJob=True for all CareerString contained in career_jobs_unparsed_uncoded
- don't code CareerStringType - because we still have MultipleSubstrings
- just note that all career_jobs_unparsed_uncoded are contained in careers
- later, use the files in subpath_2_2 = "2.2 career_divided_unparsed_uncoded/" to help code CareerSubstringType

In [22]:
# compare careers with careerjobs
careerjobs.head(3)

,CareerString,CareerDateString_2022,CareerStringID
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,1
1,근로단체부장 해임 * 후임: 리일환,2014,2
2,노동성 상(유임),1999.02,4


In [23]:
select_columns = ["CareerString","CareerDateString_2022"]
careerjobs = careerjobs[select_columns]
careerjobs.shape

(7590, 2)

In [24]:
careerjobs[careerjobs.duplicated(keep=False)]

,CareerString,CareerDateString_2022


In [25]:
careers.head(3)

,CareerString,CareerDateString_2022
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,故 김정일 국가장의위원회 위원,2011.12
2,근로단체부장 해임 * 후임: 리일환,2014


In [26]:
# merge and compare careers & careerjobs
select_columns = ["CareerString","CareerDateString_2022"]
careers0 = careers.merge(careerjobs,on=select_columns,how="outer",indicator=True)

In [53]:
careers0.shape

(8045, 3)

In [27]:
careers0.head(3)

,CareerString,CareerDateString_2022,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,both
1,故 김정일 국가장의위원회 위원,2011.12,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,both


In [28]:
careers0[careers0["_merge"]=="left_only"].shape

(455, 3)

In [29]:
careers0[careers0["_merge"]=="both"].shape

(7590, 3)

In [30]:
careers0[careers0["_merge"]=="right_only"].shape

(0, 3)

In [31]:
careers0[careers0["_merge"]=="right_only"]

,CareerString,CareerDateString_2022,_merge


In [32]:
careers0["IsJob"] = False

In [33]:
careers0.loc[careers0["_merge"]=="both","IsJob"] = True

In [34]:
# 7590 jobs
# 455 not jobs
careers0.groupby("IsJob").count()

,CareerString,CareerDateString_2022,_merge
IsJob,,,
False,455,259,455
True,7590,4209,7590


In [35]:
select_columns = ["CareerString","CareerDateString_2022","IsJob"]
careers0 = careers0[select_columns]

In [36]:
careers0.head(3)

,CareerString,CareerDateString_2022,IsJob
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,True
1,故 김정일 국가장의위원회 위원,2011.12,False
2,근로단체부장 해임 * 후임: 리일환,2014,True


### 2.2.5 - update CareerStringType for NotJob, using 2.2 career_divided_unparsed_uncoded

In [88]:
# not urgent. do this later. or do it again by hand

### 2.3 - careers1 - add MultipleString
- compare careers0 with jol1 (joborglink_1_mergeold_mail)
 - ignore CareerDateString_2022 and only merge on CareerString because CareerDateString_2022 is incorrect
- update IsJob with jol1["NotJob"]
- add MultipleSubstrings
- code MultipleSubstrings

In [97]:
select_columns = ["CareerString","NotJob","MultipleSubstrings"]
jol1 = jol1[select_columns]
jol1.head(3)

,CareerString,NotJob,MultipleSubstrings
0,"1992.12 정무원 총리 해임, 당 정치국 후보위원으로 강등, 자강도 당 책임비서...",NaN,2.0
1,2003. 3 조선일본군위안부 및 강제연행피해자보상대책위원장,NaN,NaN
2,"당 정치국 위원, 당 중앙위원회 부위원장, 당 국제부장 등 해임(추정)",0.0,NaN


In [98]:
jol1.shape

(7592, 3)

In [99]:
jol1 = jol1.drop_duplicates(keep="first")
jol1.shape

(6659, 3)

In [100]:
# some duplicate CareerStrings with different dates; some are legit, others not
# I removed CareerDateString_2022 from jol1 as it's not critical to updating IsJob & MultipleSubstrings
jol1[jol1.duplicated(["CareerString"],keep=False)].head(10)

,CareerString,NotJob,MultipleSubstrings


In [101]:
# no duplicate keys in jol1
select_key = ["CareerString"]
jol1[jol1.duplicated(select_key,keep=False)]

,CareerString,NotJob,MultipleSubstrings


In [102]:
careers0.shape

(8045, 3)

In [103]:
select_key = ["CareerString"]
careers1 = careers0.merge(jol1,on=select_key,how="outer",indicator=True)
careers1.shape

(8045, 6)

In [104]:
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,True,1.0,NaN,both
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN,NaN,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,True,1.0,NaN,both


In [105]:
# 7590 IsJob
# 455 NotJob
careers1.groupby(["_merge"],dropna=False).count()

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings
_merge,,,,,
left_only,455,259,455,0,0
right_only,0,0,0,0,0
both,7590,4209,7590,926,738


In [106]:
careers1[careers1["_merge"]=="right_only"]

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings,_merge


In [107]:
# NobJob = 0: 후임, etc.
# NotJob = 1: marriage, education, etc.
# NotJob = NaN: IsJob
careers1.groupby(["IsJob","NotJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  _merge
IsJob NotJob                                                                 
False NaN              455                    259                   0     455
True  0.0              422                    239                   0     422
      1.0              504                    203                   0     504
      NaN             6664                   3767                 738    6664

In [108]:
# if NotJob in [0,1], set IsJob = False
careers1.loc[careers1["NotJob"].isin([0,1]),"IsJob"]=False

In [109]:
# verify successful update of IsJob
careers1.groupby(["IsJob","NotJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  _merge
IsJob NotJob                                                                 
False 0.0              422                    239                   0     422
      1.0              504                    203                   0     504
      NaN              455                    259                   0     455
True  NaN             6664                   3767                 738    6664

In [110]:
# NobJob = 0: 후임, etc.
# NotJob = 1: marriage, education, etc.
# NotJob = NaN: IsJob
careers1.groupby(["MultipleSubstrings"],dropna=False).count()

,CareerString,CareerDateString_2022,IsJob,NotJob,_merge
MultipleSubstrings,,,,,
2.0,574,257,574,0,574
3.0,123,70,123,0,123
4.0,27,15,27,0,27
5.0,14,10,14,0,14
NaN,7307,4116,7307,926,7307


In [116]:
# accept MultipleSubstrings
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,NaN
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN
2,근로단체부장 해임 * 후임: 리일환,2014,False,NaN


In [118]:
# drop NotJob, _merge
careers1.drop(columns=["NotJob","_merge"],inplace=True)
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,NaN
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN
2,근로단체부장 해임 * 후임: 리일환,2014,False,NaN


In [119]:
careers1.groupby(["IsJob","MultipleSubstrings"],dropna=False).count()

CareerString  CareerDateString_2022
IsJob MultipleSubstrings                                     
False NaN                         1381                    701
True  2.0                          574                    257
      3.0                          123                     70
      4.0                           27                     15
      5.0                           14                     10
      NaN                         5926                   3415

### 2.4 - careers2 - add CareerSubstrings

- compare careers1 with
 - joborglink_2_substr_single_1tocode_4Mail_Lee
 - joborglink_2_substr_multi2_1tocode_Mail_Lee
- double-check
 - CareerString: matches careers or jol1 (incomplete)?
 - CareerDateString_2022: matches careers, or jol1 (unreliable)?
- add CareerStartYear, CareerStartMonth, CareerSubstrings

In [152]:
select_columns = ["CareerString","CareerDateString_2022","CareerStartYear","CareerStartMonth"]
singlesub = singlesub[select_columns]
singlesub.shape

(5897, 4)

In [153]:
singlesub.head(3)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,2013,9.0
1,근로단체부장 해임 * 후임: 리일환,2014,2014,NaN
2,노동성 상(유임),1999.02,1999,2.0


In [154]:
# 
singlesub[singlesub.duplicated(keep=False)].shape

(42, 4)

In [155]:
singlesub = singlesub.drop_duplicates(keep="first")
singlesub.shape

(5873, 4)

In [158]:
singlesub[singlesub.duplicated("CareerString",keep=False)]

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth
14,(前)당 정치국 위원,2021.01,2021,1.0
15,(前)당 정치국 위원,연도미상,NaN,NaN
1522,2.8직동청년탄광 지배인,연도미상,NaN,NaN
1523,2.8직동청년탄광 지배인,1999.02,NaN,NaN
2405,3군단장,1957.10,1957,NaN
...,...,...,...,...
5849,화학공업상(유임),2019.04,2019,4.0
5856,황해남도 인민위원회 위원장,2006.06 ~ 2012.03,2006,6.0
5857,황해남도 인민위원회 위원장,1999.06,1999,6.0
5879,황해제철연합기업소 지배인,1987.11,1987,11.0


In [159]:
select_columns = ["CareerString","CareerDateString_2022","CareerStartYear","CareerStartMonth","MultipleSubstrings","CareerSubstring"]
multisub = multisub[select_columns]
multisub.shape

(1756, 6)

In [160]:
multisub.head(3)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstring
0,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN
1,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN
2,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,2012,4.0,2,NaN


In [162]:
multisub[multisub.duplicated(keep=False)].head(5)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstring
0,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN
1,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN
2,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,2012,4.0,2,NaN
3,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,2012,4.0,2,NaN
4,△인민군 총참모장 △인민군 대장,2009.02,2009,2.0,2,NaN
